# Importando los paquetes que vamos a usar

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

# Carga de los datos

In [2]:
base_path = "dataset/datos_hogar_2020.csv"
df_global = pd.read_csv(base_path)

# Funciones comunes a todos los casos de estudio

Definimos en esta sección las funciones que vamos a usar a lo largo de todos los casos de estudio:

In [3]:
# TODO -- añadir algo aqui

In [4]:
df_global.describe()

,DB030,DB040_F,DB060,DB060_F,DB090,DB090_F,DB100,DB100_F,HB050,HB050_F,...,HC050,HC050_F,HV010,HV010_F,HV070,HV070_F,HV020,HV020_F,HV080,HV080_F
count,15043.000000,15043.0,15043.000000,15043.0,1.504300e+04,15043.0,15043.000000,15043.0,15043.000000,15043.0,...,15005.000000,15043.000000,8.346000e+03,15043.000000,3281.000000,15043.000000,15008.000000,15043.000000,14758.000000,15043.000000
mean,12025.060693,1.0,2369.109420,2.0,1.248016e+08,1.0,1.791199,1.0,10.872765,1.0,...,2.479440,0.994948,1.903000e+05,-0.102440,75231.917098,-1.290899,1.649120,0.995347,2.223879,0.962109
std,7836.885487,0.0,1722.686793,0.0,1.468081e+08,0.0,0.852394,0.0,1.200997,0.0,...,1.398692,0.100397,2.134496e+05,1.275065,63819.140803,1.230864,0.477261,0.096362,1.195140,0.272676
min,1.000000,1.0,1.000000,2.0,1.250433e+06,1.0,1.000000,1.0,1.000000,1.0,...,1.000000,-1.000000,3.000000e+03,-2.000000,104.000000,-2.000000,1.000000,-1.000000,1.000000,-1.000000
25%,5279.500000,1.0,860.000000,2.0,4.248612e+07,1.0,1.000000,1.0,10.000000,1.0,...,1.000000,1.000000,1.000000e+05,-1.000000,30000.000000,-2.000000,1.000000,1.000000,1.000000,1.000000
50%,10751.000000,1.0,1722.000000,2.0,8.289029e+07,1.0,2.000000,1.0,11.000000,1.0,...,2.000000,1.000000,1.500000e+05,1.000000,60000.000000,-2.000000,2.000000,1.000000,2.000000,1.000000
75%,18695.500000,1.0,4052.000000,2.0,1.481213e+08,1.0,3.000000,1.0,11.000000,1.0,...,4.000000,1.000000,2.300000e+05,1.000000,100000.000000,-1.000000,2.000000,1.000000,3.000000,1.000000
max,26816.000000,1.0,5232.000000,2.0,2.773637e+09,1.0,3.000000,1.0,12.000000,1.0,...,4.000000,1.000000,7.000000e+06,1.000000,525000.000000,1.000000,2.000000,1.000000,4.000000,1.000000


# Filtrado previo del dataset global

Definimos en variables todas las columnas que, al leer el documento explicando el *dataset*, nos han resultado de interés. Una vez hecho esto, filtramos el *dataset* global para quedarnos solo con esas columnas, pues el resto no las vamos a usar en ningún caso de estudio. Es más, muchas de estas columnas no las usaremos en ninguno de los tres casos de estudio.

In [5]:
# Definimos las columnas con las que nos vamos a quedar
renta_disponible_total_hogar = "HY020"
renta_ganada_alquilando = "HY040N"
ayuda_para_vivienda = "HY070N"
inversiones_de_capital = "HY090N"
retraso_pago_hipoteca_o_alquiler = "HS011"
retraso_pago_luz_agua_gas = "HS021"
retraso_pago_prestamos = "HS031"
capacidad_hogar_llegar_fin_mes = "HS120"
ingresos_minimos_para_llegar_fin_mes = "HS130"
vandalismo_en_la_zona = "HS190"

tipo_de_vivienda = "HH010"
regimen_de_tenencia_vivienda = "HH021"
habitaciones_vivienda = "HH030"

pidio_ayuda_economica_familiares = "H39A_U"
pidio_ayuda_economica_entidad = "H39B_U"
corte_energia = "H38A_U"

numero_miembros_hogar = "HX040"
tipo_de_hogar = "HX060" # Variable numerica categorica con muchos valores posibles
hogar_en_riesgo_pobreza = "vhPobreza"
hogar_carencia_material_severa = "vhMATDEP"

proposito_prestamos = lambda i: f"HI100_{i}"

# Filtramos todo el dataframe global para usar solo variables que en una primera
# lectura de las variables hemos considerado interesantes
df_global = df_global[[
    renta_disponible_total_hogar,
    renta_ganada_alquilando,
    ayuda_para_vivienda,
    inversiones_de_capital,
    retraso_pago_hipoteca_o_alquiler,
    capacidad_hogar_llegar_fin_mes,
    ingresos_minimos_para_llegar_fin_mes,
    vandalismo_en_la_zona,
    tipo_de_vivienda,
    regimen_de_tenencia_vivienda,
    habitaciones_vivienda,   
    pidio_ayuda_economica_familiares,
    pidio_ayuda_economica_entidad,
    corte_energia,
    numero_miembros_hogar,
    tipo_de_hogar,
    hogar_en_riesgo_pobreza,
    hogar_carencia_material_severa,
    
    # TODO -- borrar los propositos porque no parecen muy relevantes
    proposito_prestamos(1),
    proposito_prestamos(2),
    proposito_prestamos(3),
    proposito_prestamos(4),
    proposito_prestamos(5),
    proposito_prestamos(6),
    proposito_prestamos(7),
    proposito_prestamos(8),
    proposito_prestamos(9),


]]
df_global

,HY020,HY040N,HY070N,HY090N,HS011,HS120,HS130,HS190,HH010,HH021,...,vhMATDEP,HI100_1,HI100_2,HI100_3,HI100_4,HI100_5,HI100_6,HI100_7,HI100_8,HI100_9
0,0.00,0.0,0.0,0.0,NaN,5.0,1500.0,2.0,3.0,1,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,13221.50,0.0,0.0,0.0,NaN,1.0,1000.0,2.0,3.0,1,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,24500.00,0.0,0.0,0.0,3.0,5.0,1500.0,2.0,4.0,2,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,16033.10,0.0,0.0,0.0,3.0,3.0,1500.0,2.0,4.0,2,...,0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
4,39660.00,0.0,0.0,0.0,3.0,4.0,3000.0,2.0,4.0,2,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15038,39969.70,0.0,0.0,0.0,3.0,2.0,2200.0,2.0,2.0,1,...,0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
15039,66003.88,2500.0,0.0,684.9,NaN,5.0,2000.0,2.0,1.0,5,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15040,24702.60,0.0,0.0,0.0,NaN,3.0,1800.0,1.0,1.0,1,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15041,5564.10,0.0,0.0,0.0,NaN,2.0,900.0,2.0,3.0,5,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Caso de estudio 01

## Definición del caso de estudio

En este caso de estudio queremos poner nuestra atenció en ... TODO -- añadir aqui contenido